# This notebook
This notebook aims to submit the pretrained DT model from awesome notebook https://www.kaggle.com/lebroschar/1000-greedy-decision-tree-model.

I refered discussions like https://www.kaggle.com/c/santa-2020/discussion/203427 and https://www.kaggle.com/c/halite/discussion/177686 to make this notebook.

If this notebook is useful for you, please upvote these notebooks and discussions.

# Simple Decision Tree model to predict payoff probability

Consistently beats Bayesian UCB, Simple Multi-armed Bandit, and Thompson sampling kernels (see below).

**NOTE:** This won't work as a submission as is, since it is referencing the training data file (I think).  Please see the post from Sirith Somanchi about how to make it work as a submission using base64:
https://www.kaggle.com/c/santa-2020/discussion/203427

In [ ]:
"""Greedy agent that chooses machine based on maximum expected payout

Uses a trained decision tree model to consider the other player's movements
in the expected payout.

See my other kernel for methodology for generating training data:
https://www.kaggle.com/lebroschar/generate-training-data

"""
import pickle
import base64
import random


import numpy as np
import pandas as pd
import sklearn.tree as skt

# Parameters
FUDGE_FACTOR = 0.99
VERBOSE = False
DATA_FILE = '/kaggle/input/sample-training-data/training_data_201223.parquet'
TRAIN_FEATS = ['round_num', 'n_pulls_self', 'n_success_self', 'n_pulls_opp']
TARGET_COL = 'payout'



def make_model():
    """Builds a decision tree model based on stored trainingd data"""
    data = pd.read_parquet(DATA_FILE)
    model = skt.DecisionTreeRegressor(min_samples_leaf=40)
    model.fit(data[TRAIN_FEATS], data[TARGET_COL])
    return model

# Train model and save as a sav file.
model = make_model()
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
%%writefile main.py

"""Greedy agent that chooses machine based on maximum expected payout

Uses a trained decision tree model to consider the other player's movements
in the expected payout.

See my other kernel for methodology for generating training data:
https://www.kaggle.com/lebroschar/generate-training-data

"""
import pickle
import base64
import random


import numpy as np
import pandas as pd
import sklearn.tree as skt
import sys
import os

# Below is needed to submit tar.gz file to Kaggle.
sys.path.append("/kaggle_simulations/agent")
working_dir = "/kaggle_simulations/agent"
path_to_model = os.path.join(working_dir,"model.sav")

# Parameters
FUDGE_FACTOR = 0.99
VERBOSE = False
DATA_FILE = '/kaggle/input/sample-training-data/training_data_201223.parquet'
TRAIN_FEATS = ['round_num', 'n_pulls_self', 'n_success_self', 'n_pulls_opp']
TARGET_COL = 'payout'
filename = 'model.sav'


class GreedyStrategy:
    """Implements strategy to maximize expected value

    - Tracks estimated likelihood of payout ratio for each machine
    - Tracks number of pulls on each machine
    - Chooses machine based on maximum expected value
    
    
    """
    def __init__(self, name, agent_num, n_machines):
        """Initialize and train decision tree model

        Args:
           name (str):   Name for the agent
           agent_num (int):   Assigned player number
           n_machines (int):   number of machines in the game
        
        """
        # Record inputs
        self.name = name
        self.agent_num = agent_num
        self.n_machines = n_machines
        
        # Initialize distributions for all machines
        self.n_pulls_self = np.array([0 for _ in range(n_machines)])
        self.n_success_self = np.array([0. for _ in range(n_machines)])
        self.n_pulls_opp = np.array([0 for _ in range(n_machines)])

        # Track other players moves
        self.opp_moves = []
        
        # Track winnings
        self.last_reward_count = 0

        # Load model from other file
        self.model = pickle.load(open(path_to_model, 'rb'))
        
        # Predict expected reward
        features = np.zeros((self.n_machines, 4))
        features[:, 0] = len(self.opp_moves)
        features[:, 1] = self.n_pulls_self
        features[:, 2] = self.n_success_self
        features[:, 3] = self.n_pulls_opp
        self.predicts = self.model.predict(features)
        

    def __call__(self):
        """Choose machine based on maximum expected payout

        Returns:
           <result> (int):  index of machine to pull
        
        """
        # Otherwise, use best available
        est_return = self.predicts
        max_return = np.max(est_return)
        result = np.random.choice(np.where(
            est_return >= FUDGE_FACTOR * max_return)[0])
        
        if VERBOSE:
            print('  - Chose machine %i with expected return of %3.2f' % (
                int(result), est_return[result]))

        return int(result)
    
        
    def updateDist(self, curr_total_reward, last_m_indices):
        """Updates estimated distribution of payouts"""
        # Compute last reward
        last_reward = curr_total_reward - self.last_reward_count
        self.last_reward_count = curr_total_reward
        if VERBOSE:
            print('Last reward: %i' % last_reward)

        if len(last_m_indices) == 2:
            # Update number of pulls for both machines
            m_index = last_m_indices[self.agent_num]
            opp_index = last_m_indices[(self.agent_num + 1) % 2]
            self.n_pulls_self[m_index] += 1
            self.n_pulls_opp[opp_index] += 1

            # Update number of successes
            self.n_success_self[m_index] += last_reward
            
            # Update opponent activity
            self.opp_moves.append(opp_index)

            # Update predictions for chosen machines
            self.predicts[[opp_index, m_index]] = self.model.predict([
                [
                    len(self.opp_moves),
                    self.n_pulls_self[opp_index],
                    self.n_success_self[opp_index],
                    self.n_pulls_opp[opp_index]
                ],
                [
                    len(self.opp_moves),
                    self.n_pulls_self[m_index],
                    self.n_success_self[m_index],
                    self.n_pulls_opp[m_index]
                ]])
            

def agent(observation, configuration):
    global curr_agent
    
    if observation.step == 0:
        # Initialize agent
        curr_agent = GreedyStrategy(
            'Mr. Agent %i' % observation['agentIndex'],
            observation['agentIndex'],
            configuration['banditCount'])
    
    # Update payout ratio distribution with:
    curr_agent.updateDist(observation['reward'], observation['lastActions'])

    return curr_agent()


In [ ]:
# Make a tar.gz file for submission
!tar cvfz main.py.tar.gz main.py model.sav